<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
</center>

<h1> Práctico I - Estadística </h1>
<h3> Análisis y Visualización de Datos - 2019 </h3>

Durante este práctico vamos a trabajar sobre el dataset [Human Freedom Index 2018](https://www.cato.org/human-freedom-index-new) de el instituto Cato. Este índice mide en detalle lo que entendemos como libertad, utilizando 79 indicadores de libertad personal y económica en distintos aspectos, hasta obtener un hermoso numerito del 1 al 10. Usaremos una [versión ya limpia del dataset](https://www.kaggle.com/gsutters/the-human-freedom-index/home) que pueden descargar desde Kaggle.

Las variables más importantes sobre las que trabaja el dataset son:

* Rule of Law
* Security and Safety
* Movement
* Religion
* Association, Assembly, and Civil Society
* Expression and Information
* Identity and Relationships
* Size of Government
* Legal System and Property Rights
* Access to Sound Money
* Freedom to Trade Internationally
* Regulation of Credit, Labor, and Business

Nosotros centrarermos nuestro análisis en variables relacionadas a *Identity and Relationships* en paises de Latinoamérica, y los compararemos con las estadísticas globales. La pregunta a responder es simple: **¿Qué niveles de libertad se viven en Latinoamérica, especificamente en cuanto libertades de indentidad?**. Sin embargo, para hacer un análisis de los datos tenemos que platear también estas sub preguntas:

1. ¿Qué significa tener un puntaje de 4.5? Hay que poner los puntajes de la región en contexto con los datos del resto del mundo.
2. ¿Cuál es la tendencia a lo largo de los años? ¿Estamos mejorando, empeorando?
3. En este estudio, la libertad se mide con dos estimadores principales: *hf_score* que hace referencia a Human Freedom, y *ef_score* que hace referencia a Economic Freedom. Estos dos estimadores, ¿se relacionan de la misma manera con la libertad de identidad?

Inicialmente, en toda exploración de datos tenemos muy poca información a priori sobre el significado de los datos y tenemos que empezar por comprenderlos. Les proponemos los siguientes ejercicios como guía para comenzar esta exploración.

In [1]:
import matplotlib.pyplot as plt
import numpy
import pandas
import seaborn

In [2]:
seaborn.__version__

'0.9.0'

In [3]:
dataset = pandas.read_csv('C:/Users/nbcorar71/Documents/GitHub/diplodatos_famaf_2019/datasets/hfi_cc_2018.csv')
dataset.shape

(1458, 123)

In [4]:
dataset.columns
# Way too many columns!

Index(['year', 'ISO_code', 'countries', 'region', 'pf_rol_procedural',
       'pf_rol_civil', 'pf_rol_criminal', 'pf_rol', 'pf_ss_homicide',
       'pf_ss_disappearances_disap',
       ...
       'ef_regulation_business_bribes', 'ef_regulation_business_licensing',
       'ef_regulation_business_compliance', 'ef_regulation_business',
       'ef_regulation', 'ef_score', 'ef_rank', 'hf_score', 'hf_rank',
       'hf_quartile'],
      dtype='object', length=123)

In [5]:
dataset[:2]

,year,ISO_code,countries,region,pf_rol_procedural,pf_rol_civil,pf_rol_criminal,pf_rol,pf_ss_homicide,pf_ss_disappearances_disap,...,ef_regulation_business_bribes,ef_regulation_business_licensing,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,ef_score,ef_rank,hf_score,hf_rank,hf_quartile
0,2016,ALB,Albania,Eastern Europe,6.661503,4.547244,4.666508,5.291752,8.920429,10.0,...,4.050196,7.324582,7.074366,6.705863,6.906901,7.54,34.0,7.568140,48.0,2.0
1,2016,DZA,Algeria,Middle East & North Africa,NaN,NaN,NaN,3.819566,9.456254,10.0,...,3.765515,8.523503,7.029528,5.676956,5.268992,4.99,159.0,5.135886,155.0,4.0


Por suerte las columnas tienen un prefijo que nos ayuda a identificar a qué sección pertenecen. Nos quedamos sólo con las que comienzan con *pf_indentity*, junto con otras columnas más generales

In [6]:
important_cols = ['year', 'ISO_code', 'countries', 'region']

important_cols += [col for col in dataset.columns if 'pf_identity' in col]

important_cols += [
    'ef_score', # Economic Freedom (score)
    'ef_rank', # Economic Freedom (rank)
    'hf_score', # Human Freedom (score)
    'hf_rank', # Human Freedom (rank)
]


## 1. Estadísticos descriptivos

  1. Para comenzar con un pantallazo de los datos, calcular el rango de las variables.
  2. Obtener media, mediana y desviación estándar de las variables *pf_identity* y *hf_score* en el mundo y compararla con la de Latinoamérica y el caribe. ¿Tiene sentido calcular la moda? 
  3. ¿Son todos los valores de *pf_identity*  y *hf_score* directamente comparables? ¿Qué otra variable podría influenciarlos?
  4. ¿Cómo pueden sanearse los valores faltantes?
  5. ¿Encuentra outliers en estas dos variables? ¿Qué método utiliza para detectarlos? ¿Los outliers, son globales o por grupo? ¿Los eliminaría del conjunto de datos?
  
  

In [7]:
#--------------------------------------------------------------------------------------#
#1- Para comenzar con un pantallazo de los datos, calcular el rango de las variables.  #
#--------------------------------------------------------------------------------------#
#Agrupo solo las variables Numericas para poder calcular el rango
range_cols = ['year', 
 'pf_identity_legal',
 'pf_identity_parental_marriage',
 'pf_identity_parental_divorce',
 'pf_identity_parental',
 'pf_identity_sex_male',
 'pf_identity_sex_female',
 'pf_identity_sex',
 'pf_identity_divorce',
 'pf_identity',
 'ef_score',
 'ef_rank',
 'hf_score',
 'hf_rank']


# mostramos cantida de registros, cantidad de valores unicos y rangos para cada variable
descript_rango = pandas.DataFrame([dataset[range_cols].count(),dataset[range_cols].nunique(),dataset[range_cols].min(), dataset[range_cols].max(), dataset[range_cols].max() - dataset[range_cols].min()], 
                     index=['cant_reg','valores_unicos','min','max','rango']).transpose()
descript_rango

,cant_reg,valores_unicos,min,max,rango
year,1458.0,9.0,2008.000000,2016.000000,8.000000
pf_identity_legal,205.0,3.0,0.000000,10.000000,10.000000
pf_identity_parental_marriage,923.0,3.0,0.000000,10.000000,10.000000
pf_identity_parental_divorce,923.0,3.0,0.000000,10.000000,10.000000
pf_identity_parental,1358.0,5.0,0.000000,10.000000,10.000000
pf_identity_sex_male,1375.0,3.0,0.000000,10.000000,10.000000
pf_identity_sex_female,1378.0,3.0,0.000000,10.000000,10.000000
pf_identity_sex,1378.0,3.0,0.000000,10.000000,10.000000
pf_identity_divorce,585.0,3.0,0.000000,10.000000,10.000000
pf_identity,1378.0,30.0,0.000000,10.000000,10.000000


1.1) Mostramos la cantidad de registros no nulos [cant_reg], la cantidad de valores distintos por columnas [valores_unicos],  los valores minimos [min] y  vzlores maximos [max] y los rangos de cada una de las variables [rango]

In [20]:
#----------------------------------------------------------------------------------------------------------#
#  2- Obtener media, mediana y desviación estándar de las variables pf_identity y hf_score en el mundo     #
#  y compararla con la de Latinoamérica y el caribe. ¿Tiene sentido calcular la moda?                      #
#----------------------------------------------------------------------------------------------------------#
datos_latinoamerica = dataset[dataset['region'] == 'Latin America & the Caribbean']
datos_resto_mundo = dataset[dataset['region'] != 'Latin America & the Caribbean']

med_dev_std_cols = [ 'pf_identity', 'hf_score']
descript_med_dev_std = pandas.DataFrame([datos_resto_mundo[med_dev_std_cols].median(),
                                         datos_latinoamerica[med_dev_std_cols].median(),
                                         datos_resto_mundo[med_dev_std_cols].mean(),
                                         datos_latinoamerica[med_dev_std_cols].mean(),
                                         datos_resto_mundo[med_dev_std_cols].std(),
                                         datos_latinoamerica[med_dev_std_cols].std()                                         
                                        ], 
                     index=['mediana_rest_m','mediana_la','media_rest_m','media_la','dev_std_rest_m','dev_std_rest_la']).transpose()
descript_med_dev_std

,mediana_rest_m,mediana_la,media_rest_m,media_la,dev_std_rest_m,dev_std_rest_la
pf_identity,8.333333,10.000000,7.044503,8.765086,3.303771,1.711732
hf_score,6.867191,7.028045,6.982403,7.047982,1.090861,0.608904


1.2) Para la variable pf_identity podriamos calcularle la moda, pero para el caso de la variable hf_score no tiene sentido calcular la moda ya que son valores unicos.

1.3) Se me complica decir correctamente si las variables *pf_identity*  y *hf_score* son directamente comparables entiendo que no deberian compararse ya que la variable pf_identity se utiliza para el calculo de la hf_score.

1.4)¿Cómo pueden sanearse los valores faltantes? para sanearse los valores faltantes podemos utilizar la funcion dropna(), pero esta funcion estaria eliminando los registros que sean nulos tambien podemos imputar los valores numericos por la media o la mediana, en principio cuando analizamos la variable deberiamos descartar aquellas variables que posean mas del 20% de valores nulos.

 5. ¿Encuentra outliers en estas dos variables? ¿Qué método utiliza para detectarlos? ¿Los outliers, son globales o por grupo? ¿Los eliminaría del conjunto de datos?
  

## 2. Agregación de datos

1. Grafiquen la media de la variable *pf_identity* y *hf_score* a través de los años.
2. Realicen los mismos gráficos, pero separando por regiones (Cada variable en un gráfico distinto, sino no se ve nada). ¿La tendencia observada, es la misma que si no dividimos por regiones?
3. Si lo consideran necesario, grafiquen algunos países de Latinoamerica para tratar de explicar la tendencia de la variable *pf_identity* en la región. ¿Cómo seleccionarion los países relevantes a esa tendencia?

Hint: hay un gráfico de seaborn que hace todo por vos!

Sólo por curiosidad, graficar la tendencia de *hf_score* y *ef_score* a través de los años. ¿Tienen alguna hipótesis para este comportamiento?

## 2. Distribuciones
  1. Graficar en un mismo histograma la distribución de la variable *pf_identity* en global, y en Latinoamérica y el caribe. Repetir para la variable *hf_score*. ¿Visualmente, a qué tipo de distribución corresponde cada variable? ¿Es correcto utilizar todos los registros para esas zonas en estos gráficos?
  2. Realizar una prueba de Kolmogorov-Smirnof para comprobar analíticamente si estas variables responden la distribución propuesta en el ejercicio anterior. Hint: podés usar https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.kstest.html, pero hay que tener en cuenta que si la distribución es "norm", entonces va a comparar los datos con una distribución normal con media 0 y desviación estándar 1. Se puede utilizar la distribución sobre todos los datos o sólo sobre Latinoamérica.
  3. Realizar un gráfico QQ de las mismas distribuciones. Se puede utilizar a,bas distribuciones sobre todos los datos o sólo sobre Latinoamérica, pero no cruzadas.
  4. Medir la asimetría (skew) y curtosis de las mismas distribuciones con las que realizó el gráfico anterior. ¿Cómo se relacionan estos estadísticos con la forma del gráfico QQ obtenido previamente? ¿El gráfico QQ provee más información que no esté presente en estos estadísticos?

## 3. Correlaciones

En este ejercicio queremos responder a las preguntas

* Las libertades sociales y económicas, ¿van siempre de la mano?
* ¿Cómo se relacionan ambas con las libertades individuales y respectivas a las relaciones personales?

Para ello, analizaremos las correlaciones entre las variables pf_identity, hf_score y ef_score. 

Como pf_indentity contribuye al cálculo de hf_score y ef_score, esperamos hallar algún grado de correlación. Sin embargo, queremos medir qué tanta correlación. 

1. ¿Qué conclusiones puede sacar de un gráfico pairplot de estas tres variables? ¿Es adecuado para los valores de pf_identity? ¿Por qué?
2. Graficar la correlación entre pf_identity y hf_score; y entre pf_identity y ef_score. Analizar el resultado, ¿se pueden sacar conclusiones? Tengan en cuenta que como pf_identity es el resultado de un promedio, sólo toma algunos valores. Es, en efecto, discreta.
3. Calcular algún coeficiente de correlación adecuado entre los dos pares de variables, dependiendo de la cantidad de datos, el tipo de datos y la distribución de los mismo. Algunas opciones son: coeficiente de pearson, coeficiente de spearman, coeficientes de tau y de kendall. Interpretar los resultados y justificar si las variables están correlacionadas o no. 
4. [Opcional] Analizar la correlación entre la region y el hf_score (y/o el ef_score); y entre la region y el pf_identity. Considerar que como la variable *region* es ordinal, debe utilizarse algún tipo de test. Explicar cuáles son los requisitos necesarios para la aplicación de ese test. (Si no se cumplieran, se pueden agregar algunos datos para generar más registros)
